## Github Coursera Capston
This notebook will be mainly used for the capstone project.

#### Assignment 1 - Capstone Project Notebook

In [11]:
import pandas as pd
import numpy as np

In [12]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


#### Assignment 3 - Segmenting and Clustering Neighborhoods in Toronto

**Download List_of_postal_codes_of_Canada and read into data frame, and then rename column Postcode to PostalCode**

In [103]:
from pandas.io.html import read_html
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(url,  attrs={"class":"wikitable"},header =0)
wiki_df = pd.DataFrame(wikitable[0])
wiki_df= wiki_df.rename(columns={'Postcode':'PostalCode'})
wiki_df.head(11)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


 **Ignore cells with a borough that is Not assigned.**

In [104]:
wiki_df_filtered_na = wiki_df[wiki_df['Borough']!="Not assigned"]
wiki_df_filtered_na.shape

(210, 3)

 **Combine rows group by postalcode and neighborhoods separated  with a comma**

In [105]:
wiki_group = wiki_df_filtered_na.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
print(wiki_group.shape)

(103, 3)


In [106]:
wiki_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [48]:
## Manipulate N/A value of Neightborhood by using Borough
wiki_group.loc[wiki_group["Neighborhood"]=="Not assigned", "Neighborhood"] = wiki_group.loc[wiki_group["Neighborhood"]=="Not assigned", "Borough"]
wiki_group.shape

(103, 3)

In [69]:
latlong = pd.read_csv("Geospatial_Coordinates.csv")
latlong=latlong.rename(columns={'Postal Code':'PostalCode'})
latlong.head()


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [71]:
wiki_group_geo=pd.merge(wiki_group, latlong, on='Postcode')
wiki_group_geo.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [72]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [73]:
geolocator = Nominatim()
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [82]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
 
for lat, long, post, borough, neigh in zip(wiki_group_geo['Latitude'], wiki_group_geo['Longitude'], wiki_group_geo['Postcode'], wiki_group_geo['Borough'], wiki_group_geo['Neighborhood']):
    label = '{}, {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#00FF00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto